In [10]:
from pyspark.sql import SparkSession
from great_expectations.dataset.sparkdf_dataset import SparkDFDataset
from great_expectations.data_context import DataContext
import great_expectations as gx
# Initialize Spark Session
spark = SparkSession.builder.appName("GE with PySpark").getOrCreate()


# context = gx.get_context()
context = DataContext("gx/")

datasource = context.sources.add_spark("tst")
name = "my_df_dataset"
data_asset = datasource.add_dataframe_asset(name=name)


# Load your data into a PySpark DataFrame
df = spark.read.csv("data/train/train-1.csv", header=True, inferSchema=True)


batch_request = data_asset.build_batch_request(dataframe=df)


# # Convert the PySpark DataFrame to a Great Expectations Dataset
# ge_df = SparkDFDataset(df)

# # Load the DataContext and expectation suite
# context = DataContext("great_expectations")
# suite = context.get_expectation_suite("your_expectation_suite_name")

# # Validate the DataFrame against the suite
# results = context.run_validation_operator("action_list_operator", assets_to_validate=[ge_df], expectation_suite_name=suite.expectation_suite_name)

# # Print the validation results
# print(results)


DataContextError: Can not write the fluent datasource tst because a datasource of that name already exists in the data context.

In [3]:
batch_request

BatchRequest(datasource_name='tst', data_asset_name='my_df_dataset', options={})

In [4]:
context.add_or_update_expectation_suite("my_expectation_suite")


{
  "expectation_suite_name": "my_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.11"
  }
}

In [5]:
context

{
  "anonymous_usage_statistics": {
    "explicit_url": false,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_id": true,
    "enabled": true,
    "data_context_id": "f0ecb877-5a37-47d7-bbab-2a4c7aafaa41"
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "my_spark_datasource": {
      "type": "spark",
   

In [6]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="my_expectation_suite",
)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

24/03/19 11:41:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: _c0, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: _c0 but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train/train-1.csv


,_c0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,None,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wáyné Dyér is á pớpúlár áméricán pérsớnál grớw...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tớllé Máthé Ápp...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False


In [7]:
validator.expect_column_values_to_not_be_null(column="label")



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1200,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
validator.expectation_suite

In [8]:
validator.save_expectation_suite(discard_failed_expectations=False)

